In [16]:
import imutils  
import cv2

clicks = []
final_genes = []
genes = []
current_gene = 0
locations = []
def colony_pick(img:str,in_genes:list,in_locations:list):
    global clicks
    global final_genes
    global genes
    global current_gene
    global locations
    
    genes=in_genes
    current_gene = 0
    locations = ['Upper_left','Upper_right','Lower_left','Lower_right'] + in_locations
    clicks = []
    final_genes = []

    wells = len(locations)-4
    num_wells = wells/len(genes)
    def click_event(event, x, y, flags, param):
        global genes
        global locations
        global clicks
        global final_genes
        global current_gene

        # Confirm location
        if event == cv2.EVENT_LBUTTONDBLCLK:
            clicks.append((x,y))
            print('=======')
            if len(clicks) > 4:    
                final_genes.append(genes[current_gene])
            if final_genes.count(genes[current_gene]) == num_wells:
                current_gene+=1

            print('To pick: {}'.format(locations[len(clicks)]))
            if len(clicks) >= 4:
                print('Current gene: {}'.format(genes[current_gene]))


            if len(clicks) == len(locations):
                print('PLEASE EXIT.')

        # Next gene
        elif event == cv2.EVENT_RBUTTONUP:
            current_gene+=1
            print('Next location: {}'.format(locations[len(clicks)]))
            if len(clicks) >= 4:
                print('Current gene: {}'.format(genes[current_gene]))

        # Last gene
        elif event == cv2.EVENT_RBUTTONDBLCLK:
            current_gene-=1
            print('Next location: {}'.format(locations[len(clicks)]))
            if len(clicks) >= 4:
                print('Current gene: {}'.format(genes[current_gene]))

        # last location
        elif event == cv2.EVENT_MBUTTONDBLCLK:
            del clicks[-1]
            print('Next location: {}'.format(locations[len(clicks)]))
            if len(clicks) >= 4:
                print('Current gene: {}'.format(genes[current_gene]))

        elif event == cv2.EVENT_MBUTTONUP:
            print('Next location: {}'.format(locations[len(clicks)]))
            if len(clicks) >= 4:
                print('Current gene: {}'.format(genes[current_gene]))

    img = cv2.imread(img)
    width=800
    resized = imutils.resize(img,width=width)
    cv2.imshow('original', resized)
    print(locations[0])
    cv2.setMouseCallback("original", click_event)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    final_genes = ['Upper_left','Upper_right','Lower_left','Lower_right'] + final_genes
    # Put together output
    output_list = []
    for i in range(len(clicks)):
        dictionary = {'gene': final_genes[i],
                     'coords_relative_x': clicks[i][0],
                     'coords_relative_y': clicks[i][1],
                     'to': locations[i]}
        output_list.append(dictionary)
    return output_list

#if __name__ == "__main__":

In [18]:
file = '/home/koeng/gits/colony_picking/original.jpg'
input_genes = ['gene_{}'.format(str(i)) for i in range(24)]
input_locations = Plate().flat_columns()
colony_pick(file,input_genes,input_locations)


Upper_left
To pick: Upper_right
To pick: Lower_left
To pick: Lower_right
To pick: A1
Current gene: gene_0
To pick: B1
Current gene: gene_0
To pick: C1
Current gene: gene_0
To pick: D1
Current gene: gene_0
To pick: E1
Current gene: gene_1
To pick: F1
Current gene: gene_1
To pick: G1
Current gene: gene_1
To pick: H1
Current gene: gene_1
To pick: A2
Current gene: gene_2


[{'gene': 'Upper_left',
  'coords_relative_x': 133,
  'coords_relative_y': 134,
  'to': 'Upper_left'},
 {'gene': 'Upper_right',
  'coords_relative_x': 688,
  'coords_relative_y': 134,
  'to': 'Upper_right'},
 {'gene': 'Lower_left',
  'coords_relative_x': 128,
  'coords_relative_y': 989,
  'to': 'Lower_left'},
 {'gene': 'Lower_right',
  'coords_relative_x': 685,
  'coords_relative_y': 995,
  'to': 'Lower_right'},
 {'gene': 'gene_0',
  'coords_relative_x': 206,
  'coords_relative_y': 291,
  'to': 'A1'},
 {'gene': 'gene_0',
  'coords_relative_x': 198,
  'coords_relative_y': 299,
  'to': 'B1'},
 {'gene': 'gene_0',
  'coords_relative_x': 193,
  'coords_relative_y': 291,
  'to': 'C1'},
 {'gene': 'gene_0',
  'coords_relative_x': 190,
  'coords_relative_y': 273,
  'to': 'D1'},
 {'gene': 'gene_1',
  'coords_relative_x': 238,
  'coords_relative_y': 283,
  'to': 'E1'},
 {'gene': 'gene_1',
  'coords_relative_x': 247,
  'coords_relative_y': 274,
  'to': 'F1'},
 {'gene': 'gene_1',
  'coords_relative

In [2]:
import string
class Plate():
    def __init__(self, height:int=8, length:int=12,locations:list=[]) -> None:
        positions = []
        for letter in list(string.ascii_uppercase[0:height]):
            for number in range(length):
                positions.append((letter, number+1))
        self.height = height
        self.length = length
        if locations==[]:
            self.locations = [x[0]+str(x[1]) for x in positions]
        else:
            self.locations = locations

    def rows(self)-> list:
        return [list(g) for k, g in groupby(self.locations, key=lambda x: x[0])]
    def columns(self) -> list:
        return [[x for x in self.locations if x[1:] == str(num+1)] for num in range(self.length)]
    def flat_columns(self) -> list:
        return [item for sublist in self.columns() for item in sublist]
    def reverse_columns(self)-> list:
        return [sorted(x, reverse=True) for x in self.columns()]
    def subcolumns(self)-> list:
        if self.height%2 != 0:
            raise StandardError('Not compatible with multichannel')
        spaces = int(self.height/8)
        subcolumns=[]
        for column in self.columns():
            for space in range(spaces):
                count = space
                subcolumn=[]
                while count < len(column):
                    subcolumn.append(column[count])
                    count+=spaces
                subcolumns.append(subcolumn)
        return subcolumns

    def use_wells(self,tips:list)-> str:
        self.locations = [v for i, v in enumerate(self.locations) if v not in tips]
        return tips

    def use_number(self,num_wells:int=8,columns=None) -> str:
        if columns == None:
            columns=self.reverse_columns()
        for column in columns:
            if len(column) >= num_wells:
                return self.use_wells(column[0:num_wells])
        raise ValueError("No more tips.")
